In [1]:
from classifier import *

In [2]:
with open('constitucion_colombiana.txt', 'r') as f:
    text = f.readlines()

    f.close()
    
for line in text:
    if line == '\n':
        text.remove(line)

In [3]:
clean_text = text_remove_nl(text)
# Drop the title "Constitucion Politica de Colombia"
clean_text = clean_text[1:len(text)]

In [4]:
chapters = join_labels(clean_text, label = 'TITULO')
# for key in chapters:
#     print(key, chapters[key]['name'], chapters[key]['content'][:2])

print('\n------------------\n')
test_key = 'TITULO II\n'
test_text = chapters[test_key]['content']
print(test_text[-2])
print('\n------------------\n')

title_2 = join_labels(test_text, label = 'CAPITULO', name_importance = True)
for key in title_2:
    print(key, f"lines: {len(title_2[key]['content'])}", end='\n\n')


------------------

8. Proteger los recursos culturales y naturales del país y velar por la conservación de un ambiente sano,


------------------

CAPITULO 1
 lines: 60

CAPITULO 2
 lines: 100

CAPITULO 3
 lines: 11

CAPITULO 4
 lines: 22

CAPITULO 5
 lines: 12



In [8]:
art_list = articles_info(clean_text, debugging=False)
print('total articles = ', len(art_list))

total articles =  439


In [12]:
first=2
last_p1=31
for n in range(len(art_list[first:last_p1])):
    art = n+first
    print(f'Article No {art+1}: \n',
          art_list[art]['h1']['head'],art_list[art]['h1']['name'],
          art_list[art]['h2']['head'],art_list[art]['h2']['name'],
          art_list[art]['article']['name'],art_list[art]['article']['content'])
    print('\n--------------------------------------\n')

Article No 21: 
 TITULO II
 DE LOS DERECHOS, LAS GARANTIAS Y LOS DEBERES
 CAPITULO 1
 DE LOS DERECHOS FUNDAMENTALES
 Artículo 21 ['Artículo 21. Se garantiza el derecho a la honra. La ley señalará la forma de su protección.\n']

--------------------------------------

Article No 22: 
 TITULO II
 DE LOS DERECHOS, LAS GARANTIAS Y LOS DEBERES
 CAPITULO 1
 DE LOS DERECHOS FUNDAMENTALES
 Artículo 22 ['Artículo 22. La paz es un derecho y un deber de obligatorio cumplimiento.\n']

--------------------------------------

Article No 23: 
 TITULO II
 DE LOS DERECHOS, LAS GARANTIAS Y LOS DEBERES
 CAPITULO 1
 DE LOS DERECHOS FUNDAMENTALES
 Artículo 23 ['Artículo 23. Toda persona tiene derecho a presentar peticiones respetuosas a las autoridades por motivos de interés general o particular y a obtener pronta resolución. El legislador podrá reglamentar su ejercicio ante organizaciones privadas para garantizar los derechos fundamentales.\n']

--------------------------------------

Article No 24: 
 TIT